In [40]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder, LabelEncoder

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn import metrics
from scipy.stats import ks_2samp, chi2

from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn2pmml import sklearn2pmml, PMMLPipeline
from sklearn2pmml.decoration import CategoricalDomain

import os

import warnings
warnings.filterwarnings("ignore")

In [41]:
train = pd.read_csv('data/train.csv')
train.columns = [col.upper() for col in train.columns]
train.head()

,PASSENGERID,SURVIVED,PCLASS,NAME,SEX,AGE,SIBSP,PARCH,TICKET,FARE,CABIN,EMBARKED
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [42]:
test = pd.read_csv('data/test.csv')
test.columns = [col.upper() for col in test.columns]
test.head()

,PASSENGERID,PCLASS,NAME,SEX,AGE,SIBSP,PARCH,TICKET,FARE,CABIN,EMBARKED
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [43]:
keep_cols = [col for col in train.columns if col not in ['SURVIVED', 'PASSENGERID']]
X_train = train[['PASSENGERID'] + keep_cols]
Y_train = train['SURVIVED']

X_test = test[['PASSENGERID'] + keep_cols]

In [44]:
# 类别变量LabelEncoder
col_types = X_train[keep_cols].dtypes
num_cols = list(col_types[col_types != 'object'].index)
char_cols = list(col_types[col_types == 'object'].index)

for col in char_cols:
    lbl = LabelEncoder()
    lbl.fit(X_train[col].append(X_test[col]).astype(str))
    X_train[col] = lbl.transform(X_train[col].astype(str))
    X_test[col] = lbl.transform(X_test[col].astype(str))
    
# 导出测试集，用于pmml验证结果
X_test.to_csv(r'data/X_test.csv')

In [45]:
# 数据映射 类别性变量独热编码
mapper = DataFrameMapper(
    [(num_cols, None)] + [([char_col], [CategoricalDomain(invalid_value_treatment='as_is'),
                           OneHotEncoder(handle_unknown='ignore')]) for char_col in char_cols]
)

# 拟合mapper，用mapper转换后再经过模型，效果等同经过mapper和模型的管道
mapper.fit(X_train)

DataFrameMapper(default=False, df_out=False,
        features=[(['PCLASS', 'AGE', 'SIBSP', 'PARCH', 'FARE'], None), (['NAME'], [CategoricalDomain(invalid_value_replacement=None,
         invalid_value_treatment='as_is', missing_value_replacement=None,
         missing_value_treatment='as_is', missing_values=None,
         with_data=True, with_statisti...       dtype=<class 'numpy.float64'>, handle_unknown='ignore',
       n_values=None, sparse=True)])],
        input_df=False, sparse=False)

In [46]:
# 命令行代码，可调用jar包运行pmml
import subprocess

# sigmoid转换
sigmoid1 = lambda x: 1 / (1 + np.exp(-x))
sigmoid2 = lambda x: 1 / (1 + np.power(2.71828183, -x))

# 调用jar包运行pmml
def run_pmml_jar(command, verbose=True):
    proc = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    for line in proc.stdout:
        if verbose:
            print(line.strip().decode('gbk'))

# 比较数据的小数位
def compare_decimal(predict, col1, col2, digit=6):
    compare_decimal = predict[(predict[col1].apply(lambda x: x[:digit + 2]) != predict[col2].apply(lambda x: x[:digit + 2]))]
    return compare_decimal
            
# 差异信息
def compare_decimal_info(predict, col1, col2, max_digit=9):
    compare_decimal_info = pd.DataFrame(index=['完全一致'] + ['前%s位小数一致' % digit for digit in range(1, max_digit+1)],
                                       columns=['样本数', '占比'])
    data_count = predict.shape[0]
    compare_decimal_info.loc['完全一致', '样本数'] = data_count - compare_decimal(predict, col1, col2, digit=20).shape[0]
    for digit in range(1, max_digit+1):
        compare_decimal_info.loc['前%s位小数一致' % digit, '样本数'] = data_count - compare_decimal(predict, col1, col2, digit=digit).shape[0]
    
    compare_decimal_info['占比'] = (compare_decimal_info['样本数'] / data_count).apply(lambda x: '%.4f%%' % (x * 100))
    
    return compare_decimal_info

# 修改pmml文件得到所有叶节点和
def modify_xgb_pmml(pmml_file_in, pmml_file_out):
    with open(pmml_file_in, encoding='UTF-8') as original_pmml:
        content = original_pmml.readlines()
        
    insert_index = [i for i, x in enumerate(content) if x.find('probability(1)') != -1][0]
    output_add = []
    tab_count = content[insert_index][:content[insert_index].find('<')]
    output_add.append(tab_count + '<OutputField name="Group" optype="continuous" dataType="string" feature="transformedValue">\n')
    output_add.append(tab_count + '\t<FieldRef field="xgbValue"/>\n')
    output_add.append(tab_count + '</OutputField>\n')
    
    for line in output_add:
        insert_index += 1
        content.insert(insert_index, line)

    # 导出修改后的pmml
    with open(pmml_file_out, 'w', encoding='UTF-8') as pmml:
        for line in content:
            pmml.write(line)
            
    print('pmml修改完成，输出为叶子节点和')

# 修改pmml文件以测试
def modify_xgb_pmml_test(pmml_file_in, pmml_file_out, test):
    with open(pmml_file_in, encoding='UTF-8') as original_pmml:
        content = original_pmml.readlines()
    
    if (test == 1) or (test == 3):
        modify_index = [i for i, x in enumerate(content) if x.find('probability') != -1]
        content[modify_index[0]] = content[modify_index[0]].replace('dataType="float" ', '')
        content[modify_index[1]] = content[modify_index[1]].replace('dataType="float" ', '')
    if (test == 2) or (test == 3):
        modify_index = [i for i, x in enumerate(content) if x.find('normalizationMethod') != -1][0]
        content[modify_index] = content[modify_index].replace(' x-mathContext="float"', '')
    
    with open(pmml_file_out, 'w', encoding='UTF-8') as pmml:
        for line in content:
            pmml.write(line)
            
    print('pmml修改完成，输出为叶子节点和')

### XGBoost

In [47]:
# 训练模型 
xgb = XGBClassifier(random_state=1234)
xgb_pmml = PMMLPipeline([('mapper', mapper), ('model', xgb)])
xgb_pmml.fit(X_train, Y_train)

# 导出模型文件
# xgb_pmml._final_estimator.get_booster().dump_model(r'model_file/xgb_python.txt')
sklearn2pmml(xgb_pmml, r'model_file/xgb_pmml.pmml', with_repr=True)

#### 预测结果对比

In [48]:
# 导出python预测结果
xgb_predict_python = pd.DataFrame(xgb_pmml.predict_proba(X_test)).rename(columns={0:'proba_0', 1:'proba_1'})
xgb_predict_python['PASSENGERID'] = X_test['PASSENGERID'].values
xgb_predict_python.to_csv(r'python_predict/xgb_predict_python.csv')

In [49]:
# pmml预测概率值
run_pmml_jar(('java -jar pmml_predict/runpmml.jar data/X_test.csv model_file/xgb_pmml.pmml '
             'pmml_predict/xgb_predict_pmml.csv PASSENGERID'))

全部样本计算完成


In [50]:
# 按字符读取预测结果，否则浮点容易被自动损失精度
# 合并预测结果
xgb_predict_python = pd.read_csv(r'python_predict/xgb_predict_python.csv', dtype='object')
xgb_predict_pmml = pd.read_csv(r'pmml_predict/xgb_predict_pmml.csv', dtype='object')

predict = pd.merge(xgb_predict_python[['PASSENGERID', 'proba_1']].rename(columns={'proba_1': 'python预测结果'}),
        xgb_predict_pmml[['PASSENGERID', 'proba_1']].rename(columns={'proba_1': 'pmml预测结果'}), on='PASSENGERID')

In [51]:
compare_decimal_info(predict, 'python预测结果', 'pmml预测结果', max_digit=9)

,样本数,占比
完全一致,223,53.3493%
前1位小数一致,418,100.0000%
前2位小数一致,418,100.0000%
前3位小数一致,418,100.0000%
前4位小数一致,418,100.0000%
前5位小数一致,417,99.7608%
前6位小数一致,414,99.0431%
前7位小数一致,379,90.6699%
前8位小数一致,243,58.1340%
前9位小数一致,223,53.3493%


In [53]:
# 第5位小数和6位小数有差异的样本
compare_decimal(predict, 'python预测结果', 'pmml预测结果', digit=6)

,PASSENGERID,python预测结果,pmml预测结果
54,946,0.17738,0.17738001
94,986,0.31833997,0.31834
179,1071,0.945738,0.9457379
318,1210,0.12976198,0.129762


In [54]:
compare_decimal(predict, 'python预测结果', 'pmml预测结果', digit=7).head()

,PASSENGERID,python预测结果,pmml预测结果
30,922,0.09169989,0.091699906
54,946,0.17738,0.17738001
59,951,0.97878814,0.978788
76,968,0.09382419,0.0938242
78,970,0.13024808,0.1302481


#### 叶子节点和对比

In [55]:
xgb_predict_python_margin = X_test[['PASSENGERID']]
xgb_predict_python_margin['margin'] = xgb_pmml._final_estimator.predict(mapper.transform(X_test), output_margin=True)
xgb_predict_python_margin.to_csv(r'python_predict/xgb_predict_python_margin.csv')

modify_xgb_pmml('model_file/xgb_pmml.pmml', 'model_file/xgb_pmml_margin.pmml')
run_pmml_jar(('java -jar pmml_predict/runpmml.jar data/X_test.csv model_file/xgb_pmml_margin.pmml '
             'pmml_predict/xgb_predict_pmml_margin.csv PASSENGERID'))

pmml修改完成，输出为叶子节点和
全部样本计算完成


In [56]:
xgb_predict_python_margin = pd.read_csv(r'python_predict/xgb_predict_python_margin.csv', dtype='object')
xgb_predict_pmml_margin = pd.read_csv(r'pmml_predict/xgb_predict_pmml_margin.csv', dtype='object')

margin = pd.merge(xgb_predict_python_margin[['PASSENGERID', 'margin']].rename(columns={'margin': 'python所有叶节点和'}),
        xgb_predict_pmml_margin[['PASSENGERID', 'group']].rename(columns={'group': 'pmml所有叶节点和'}), on='PASSENGERID')

In [57]:
compare_decimal_info(margin, 'python所有叶节点和', 'pmml所有叶节点和', max_digit=9)

,样本数,占比
完全一致,418,100.0000%
前1位小数一致,418,100.0000%
前2位小数一致,418,100.0000%
前3位小数一致,418,100.0000%
前4位小数一致,418,100.0000%
前5位小数一致,418,100.0000%
前6位小数一致,418,100.0000%
前7位小数一致,418,100.0000%
前8位小数一致,418,100.0000%
前9位小数一致,418,100.0000%


In [58]:
margin[margin['python所有叶节点和'] != margin['pmml所有叶节点和']]

,PASSENGERID,python所有叶节点和,pmml所有叶节点和


In [59]:
margin.iloc[compare_decimal(predict, 'python预测结果', 'pmml预测结果', digit=6).index]

,PASSENGERID,python所有叶节点和,pmml所有叶节点和
54,946,-1.5342001,-1.5342001
94,986,-0.7614111,-0.7614111
179,1071,2.8581402,2.8581402
318,1210,-1.903065,-1.903065


#### python sigmoid运算对比

In [60]:
sigmoid = lambda x: 1 / (1 + np.exp(-x))

python_predict = pd.merge(xgb_predict_python[['PASSENGERID', 'proba_1']].rename(columns={'proba_1': 'python预测结果'}),
        xgb_predict_python_margin[['PASSENGERID', 'margin']].rename(columns={'margin': 'python所有叶节点和'}), on='PASSENGERID')

python_predict['np.float32数组sigmoid运算']= np.float32(sigmoid(np.float32(python_predict['python所有叶节点和']))).astype(str)

python_predict.loc[python_predict['python预测结果'] != python_predict['np.float32数组sigmoid运算']]

,PASSENGERID,python预测结果,python所有叶节点和,np.float32数组sigmoid运算


#### pmml sigmoid运算对比

In [61]:
modify_xgb_pmml_test('model_file/xgb_pmml.pmml', 'model_file/xgb_pmml_test.pmml', 1)
run_pmml_jar(('java -jar pmml_predict/runpmml.jar data/X_test.csv model_file/xgb_pmml_test.pmml '
             'pmml_predict/xgb_predict_pmml_test.csv PASSENGERID'))

xgb_predict_pmml_test = pd.read_csv(r'pmml_predict/xgb_predict_pmml_test.csv', dtype='object')[['PASSENGERID', 'proba_1']].rename(
    columns={'proba_1': 'pmml修改后预测结果1'})

modify_xgb_pmml_test('model_file/xgb_pmml.pmml', 'model_file/xgb_pmml_test.pmml', 2)
run_pmml_jar(('java -jar pmml_predict/runpmml.jar data/X_test.csv model_file/xgb_pmml_test.pmml '
             'pmml_predict/xgb_predict_pmml_test.csv PASSENGERID'))
xgb_predict_pmml_test = pd.merge(xgb_predict_pmml_test,
        pd.read_csv(r'pmml_predict/xgb_predict_pmml_test.csv', dtype='object')[['PASSENGERID', 'proba_1']].rename(
            columns={'proba_1': 'pmml修改后预测结果2'}), on='PASSENGERID')

modify_xgb_pmml_test('model_file/xgb_pmml.pmml', 'model_file/xgb_pmml_test.pmml', 3)
run_pmml_jar(('java -jar pmml_predict/runpmml.jar data/X_test.csv model_file/xgb_pmml_test.pmml '
             'pmml_predict/xgb_predict_pmml_test.csv PASSENGERID'))
xgb_predict_pmml_test = pd.merge(xgb_predict_pmml_test,
        pd.read_csv(r'pmml_predict/xgb_predict_pmml_test.csv', dtype='object')[['PASSENGERID', 'proba_1']].rename(
            columns={'proba_1': 'pmml修改后预测结果3'}), on='PASSENGERID')

xgb_predict_pmml_test = pd.merge(predict, xgb_predict_pmml_test, on='PASSENGERID')

pmml修改完成，输出为叶子节点和
全部样本计算完成
pmml修改完成，输出为叶子节点和
全部样本计算完成
pmml修改完成，输出为叶子节点和
全部样本计算完成


In [62]:
xgb_predict_pmml_test.iloc[compare_decimal(predict, 'python预测结果', 'pmml预测结果', digit=6).index]

,PASSENGERID,python预测结果,pmml预测结果,pmml修改后预测结果1,pmml修改后预测结果2,pmml修改后预测结果3
54,946,0.17738,0.17738001,0.1773800104856491,0.17738,0.17737999597524234
94,986,0.31833997,0.31834,0.318340003490448,0.31833997,0.3183399858916577
179,1071,0.945738,0.9457379,0.945737898349762,0.94573796,0.9457379395070129
318,1210,0.12976198,0.129762,0.12976199388504028,0.12976198,0.1297619737943394


In [63]:
xgb_predict_pmml_test[np.float64(xgb_predict_pmml_test['pmml修改后预测结果1']).astype(np.float32).astype(str) \
                      != xgb_predict_pmml_test['pmml预测结果']]

,PASSENGERID,python预测结果,pmml预测结果,pmml修改后预测结果1,pmml修改后预测结果2,pmml修改后预测结果3


In [64]:
sigmoid1 = lambda x: 1 / (1 + np.exp(-x))
sigmoid2 = lambda x: 1 / (1 + np.exp(-x, dtype=np.float32))
sigmoid3 = lambda x: 1 / (1 + np.exp(-x, dtype=np.float64))
# print('np.float32标量的默认sigmoid运算:', sigmoid1(np.float32(-1.5342001)),
#       '结果类型:', sigmoid1(np.float32(-1.5342001)).dtype)
# print('np.float32标量的32位sigmoid运算:', sigmoid2(np.float32(-1.5342001)),
#       '结果类型:', sigmoid2(np.float32(-1.5342001)).dtype)
# print('np.float32标量的64位sigmoid运算:', sigmoid3(np.float32(-1.5342001)),
#       '结果类型:', sigmoid3(np.float32(-1.5342001)).dtype)
print('np.float32数组的默认sigmoid运算:', sigmoid1(np.float32([-1.5342001]))[0],
      '结果类型:', sigmoid1(np.float32([-1.5342001]))[0].dtype)
print('np.float32数组的32位sigmoid运算:', sigmoid2(np.float32([-1.5342001]))[0],
      '结果类型:', sigmoid2(np.float32([-1.5342001]))[0].dtype)
print('np.float32数组的64位sigmoid运算:', sigmoid3(np.float32([-1.5342001]))[0],
      '结果类型:', sigmoid3(np.float32([-1.5342001]))[0].dtype, '\n')
# print('np.float64标量的默认sigmoid运算:', sigmoid1(np.float64(-1.5342001)),
#       '结果类型:', sigmoid1(np.float64(-1.5342001)).dtype)
# print('np.float64标量的32位sigmoid运算:', sigmoid2(np.float64(-1.5342001)),
#       '结果类型:', sigmoid2(np.float64(-1.5342001)).dtype)
# print('np.float64标量的64位sigmoid运算:', sigmoid3(np.float64(-1.5342001)),
#       '结果类型:', sigmoid3(np.float64(-1.5342001)).dtype)
print('np.float64数组的默认sigmoid运算:', sigmoid1(np.float64([-1.5342001]))[0],
      '结果类型:', sigmoid1(np.float64([-1.5342001]))[0].dtype)
print('np.float64数组的32位sigmoid运算:', sigmoid2(np.float32([-1.5342001]))[0],
      '结果类型:', sigmoid2(np.float32([-1.5342001]))[0].dtype)
print('np.float64数组的64位sigmoid运算:', sigmoid3(np.float64([-1.5342001]))[0],
      '结果类型:', sigmoid3(np.float64([-1.5342001]))[0].dtype)

np.float32数组的默认sigmoid运算: 0.17738 结果类型: float32
np.float32数组的32位sigmoid运算: 0.17738 结果类型: float32
np.float32数组的64位sigmoid运算: 0.17737999597524234 结果类型: float64 

np.float64数组的默认sigmoid运算: 0.1773799919316838 结果类型: float64
np.float64数组的32位sigmoid运算: 0.17738 结果类型: float32
np.float64数组的64位sigmoid运算: 0.1773799919316838 结果类型: float64


In [65]:
xgb_predict_python_test = xgb_predict_pmml_test[['PASSENGERID', 'pmml修改后预测结果3']]
xgb_predict_python_test['np.float32数组的64位sigmoid运算'] = sigmoid3(np.float32(python_predict['python所有叶节点和'])).astype(str)

In [66]:
xgb_predict_python_test.iloc[compare_decimal(predict, 'python预测结果', 'pmml预测结果', digit=6).index]

,PASSENGERID,pmml修改后预测结果3,np.float32数组的64位sigmoid运算
54,946,0.17737999597524234,0.17737999597524234
94,986,0.3183399858916577,0.3183399858916577
179,1071,0.9457379395070129,0.9457379395070129
318,1210,0.1297619737943394,0.1297619737943394


In [67]:
xgb_predict_python_test[xgb_predict_python_test['pmml修改后预测结果3'] != xgb_predict_python_test['np.float32数组的64位sigmoid运算']]

,PASSENGERID,pmml修改后预测结果3,np.float32数组的64位sigmoid运算


In [68]:
compare_decimal_info(xgb_predict_pmml_test, 'python预测结果', 'pmml修改后预测结果2', max_digit=9)

,样本数,占比
完全一致,277,66.2679%
前1位小数一致,418,100.0000%
前2位小数一致,418,100.0000%
前3位小数一致,418,100.0000%
前4位小数一致,418,100.0000%
前5位小数一致,418,100.0000%
前6位小数一致,416,99.5215%
前7位小数一致,365,87.3206%
前8位小数一致,293,70.0957%
前9位小数一致,277,66.2679%


### LightGBM

In [69]:
# 训练模型 
lgb = LGBMClassifier(random_state=1234)
lgb_pmml = PMMLPipeline([('mapper', mapper), ('model', lgb)])
lgb_pmml.fit(X_train, Y_train)

# 导出模型文件
sklearn2pmml(lgb_pmml, r'model_file/lgb_pmml.pmml', with_repr=True)

In [70]:
# 导出python预测结果
lgb_predict_python = pd.DataFrame(lgb_pmml.predict_proba(X_test)).rename(columns={0:'proba_0', 1:'proba_1'})
lgb_predict_python['PASSENGERID'] = X_test['PASSENGERID'].values
lgb_predict_python.to_csv(r'python_predict/lgb_predict_python.csv')

# pmml预测概率值
run_pmml_jar('java -jar pmml_predict/runpmml.jar data/X_test.csv model_file/lgb_pmml.pmml pmml_predict/lgb_predict_pmml.csv PASSENGERID')

全部样本计算完成


In [71]:
# 按字符读取预测结果，否则浮点容易被自动损失精度
# 合并预测结果
lgb_predict_python = pd.read_csv(r'python_predict/lgb_predict_python.csv', dtype='object')
lgb_predict_pmml = pd.read_csv(r'pmml_predict/lgb_predict_pmml.csv', dtype='object')

predict = pd.merge(lgb_predict_python[['PASSENGERID', 'proba_1']].rename(columns={'proba_1': 'python预测结果'}),
        lgb_predict_pmml[['PASSENGERID', 'proba_1']].rename(columns={'proba_1': 'pmml预测结果'}), on='PASSENGERID')

In [72]:
compare_decimal_info(predict, 'python预测结果', 'pmml预测结果', max_digit=17)

,样本数,占比
完全一致,413,98.8038%
前1位小数一致,413,98.8038%
前2位小数一致,413,98.8038%
前3位小数一致,413,98.8038%
前4位小数一致,413,98.8038%
前5位小数一致,413,98.8038%
前6位小数一致,413,98.8038%
前7位小数一致,413,98.8038%
前8位小数一致,413,98.8038%
前9位小数一致,413,98.8038%


In [73]:
predict[predict['python预测结果'] != predict['pmml预测结果']]

,PASSENGERID,python预测结果,pmml预测结果
198,1090,0.0008933515943878714,8.933515943878714E-4
211,1103,0.0005858472732236,5.858472732236E-4
253,1145,0.0006451483151224302,6.451483151224302E-4
285,1177,0.0001686246198829013,1.686246198829013E-4
415,1307,0.00013847138974055706,1.3847138974055706E-4
